# Build Convolutional Neural Network

<font color='steelblue'>

## MNIST Digit Recognition  
### From the Keras datasets, import the MNIST Digits data
#### There are images of digits 0 to 9 and have labels associated with each image<br>
<font size = "3">

**Following examples are included in the processing:**

- Check the version of `Tensorflow and Keras`
- Load `training and test` data including labels
- `Normalize` the images
- `Plot few images` after being normalized
- Create a `Neural Network` and build a model
- `Train the model` on the training dataset
- Evaluate the `accuracy of the model` using test dataset
- `Plot the accuracy and loss` for the model
</font>
</font>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [ ]:
# make sure tensorflow is properly installed
tf.__version__, tf.keras.__version__

## Locate the dataset

In [ ]:
digits_mnist = keras.datasets.mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = \
                                digits_mnist.load_data()

## Explore Data

In [ ]:
print("Size: train images {}, train labels {}".format(train_images.shape, 
                                                      train_labels.shape))

# save the number of items in training dataset
train_rows = train_images.shape[0]

In [ ]:
print("Size: test images {}, test labels {}".format(test_images.shape, 
                                                      test_labels.shape))

# save the number of items in test dataset
test_rows = test_images.shape[0]

In [ ]:
# look at first 10 labels in training set
train_labels[:10]

In [ ]:
plt.figure
plt.imshow(train_images[5], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images[0]

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

## Normalize the images

In [ ]:
# Normalize the values to be between 0 and 1; min-max normalization

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images[0]

## Plot few normalized images

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
# define the image shape (horizontal pixels x vertical pixels x grey scale)
iShape = (28,28, 1)

In [ ]:
# The data needs to be reshaped.
# The first convolution expects a single tensor containing everything, 
# so instead of 60,000 28x28x1 items in an array (60000, 28, 28), 
# it wants a single 4D array/tensor that is 60000x28x28x1, otherwise you will get an error.

train_images = train_images.reshape(train_rows, 28, 28, 1)

# reshape test images into a single tensor
test_images = test_images.reshape(test_rows, 28, 28, 1)

# Create Convolution Neural Network

In [ ]:
# Create Convolution Neural Network - adding layers

tf.random.set_seed(2345)

model = keras.models.Sequential()
model.add(layers.Conv2D(64, activation = 'relu', input_shape = iShape, 
                        kernel_size=(3, 3), padding = 'same'))

# (2,2) - pooling filter
model.add(layers.MaxPooling2D(pool_size=(2, 2)))          # (2,2) - pooling size

# (3,3) - filter size, 64 number of filters
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'valid'))   

# (2,2) - pooling filter
model.add(layers.MaxPooling2D(pool_size=(2, 2)))          # (2,2) - pooling size

In [ ]:
# To avoid over fitting on training dataset, randomly drop neurons and their connections
# remove 20% of them
model.add(layers.Dropout(rate = 0.1))

In [ ]:
# Convert the previous layer into 1 dimensional array (flatten it)
model.add(layers.Flatten())

In [ ]:
# Once we’ve flattened the data into a 1D array, add a dense hidden layer, which is normal 
# to a traditional neural network. Next, add another dropout layer before 
# adding a final dense layer which classifies the data
model.add(layers.Dense(128, activation='relu'))

#model.add(layers.Dropout(0.2))

# output layer
model.add(layers.Dense(10, activation='softmax'))   # helps classify into classes

In [ ]:
# Compile the model with chosen parameters
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Model Summary
<h3>
    <ul>
        <li>Shows all parameters and shape for each of the layers</li>
        <li>Trainable params: is total number of parameters in our model</li>
        <li>Non-trainable params: are parameters that cannot be used by model for training</li>
    </ul>
</h3>

# Train Convolutional Neural Network (CNN)
<h3><b>Used 10 epochs and got about 99.08% accuracy on training and the validation set</b></h3>

In [ ]:
%%time
# Train the model and include a validation set (composed of 10% of the dataset)
# Capturing the returned history enables you to plot the change in 
# error/loss and accuracy over time

# takes about 10 minutes 
tf.random.set_seed(2345)

batches = 1000
epochs = 6
history = model.fit(train_images, train_labels, validation_split = 0.1, 
                    batch_size = batches, epochs = epochs)

In [ ]:
metrics_names = model.metrics_names
metrics_names

# Evaluate the accuracy of the model

In [ ]:
# Use the test images to evaluate the model on a set of unseen images

test_loss, test_acc = model.evaluate(test_images, test_labels)

print("Test accuracy: ", test_acc)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string1, string2):
    # 2 rows 1 column
    plt.subplots(2, 1, sharex=False, sharey=False, figsize=(8,6))
    # plot 1
    plt.subplot(211)
    plt.plot(history.history[string1])
    plt.plot(history.history['val_'+string1])
    plt.ylabel(string1)
    plt.legend([string1, 'val_'+string1]);
    
    # plot 2
    plt.subplot(212)
    plt.plot(history.history[string2])
    plt.plot(history.history['val_'+string2])
    plt.xlabel('Epochs')
    plt.ylabel(string2)
    plt.legend([string2, 'val_'+string2]);

In [ ]:
plot_graphs(history, metrics_names[0], metrics_names[1])

In [ ]:
preds = model.predict(test_images)

In [ ]:
# preds currently has the probability for each of the 10000 test images
preds[:5]

In [ ]:
import numpy as np
# get the prediction for each image
preds = np.argmax(preds, axis = 1)

In [ ]:
preds[:5]

In [ ]:
# Print first 10 predictions
fig, axes = plt.subplots(ncols=10, sharex=False,
                         sharey=True, figsize=(20, 4))
for i in range(10):
    axes[i].set_title(preds[i])
    axes[i].imshow(test_images[i], cmap='gray')
    axes[i].get_xaxis().set_visible(False)
    axes[i].get_yaxis().set_visible(False)
plt.show()

In [ ]:
print("Number of correct predictions: {}".format(len(preds[preds == test_labels])))

In [ ]:
print("Number of incorrect predictions: {}".format(len(preds[preds != test_labels])))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, preds)

In [ ]:
import seaborn as sn
plt.figure(figsize = (8,6))
sn.heatmap(cm, annot=True, cmap=plt.cm.Blues, fmt = 'g')
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label');

## Explore incorrect predictions

In [ ]:
fig, axes = plt.subplots(ncols=10, sharex=False,
                         sharey=True, figsize=(20, 4))
count = 0
for i in range(10000):
    if preds[i] != test_labels[i]:
        axes[count].set_title(preds[i])
        axes[count].imshow(test_images[i], cmap='gray')
        axes[count].get_xaxis().set_visible(False)
        axes[count].get_yaxis().set_visible(False)
        count = count + 1
        if count == 10:
            break
plt.show()

<span style="font-family:Comic sans MS; font-size:1.4em;">
<font color='tomato'>
    <h2>Practice</h2>
    <h3>Try out different parameters and see how model accuracy changes</h3>
    <ol>
        <li>Play with different epoch values (10, 20, ...)</li>
        <li>Add more Conv2D and Pooling layers</li>
        <li>Change number of neuron in each dense layer</li>
        <li>Change the batch size and see what happens</li>    
    </ol>
</font>
</span>